In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as trans

trainSet = dsets.MNIST(root='./data', train = True, transform=trans.ToTensor(), download=True)


In [2]:
print('Number of images {}'.format(len(trainSet)))
print('Type {}'.format(type(trainSet[0][0])))
print('Size of each image {}'.format(trainSet[0][0].size()))

Number of images 60000
Type <class 'torch.Tensor'>
Size of each image torch.Size([1, 28, 28])


In [3]:
trainSet[0][0].size() # each image is of size 28 X 28 and is greyscale

torch.Size([1, 28, 28])

In [4]:
class MultiLogisticModel(nn.Module):
    def __init__(self, in_dim, out_dim):
        super(MultiLogisticModel, self).__init__()
        self.linear = nn.Linear(in_dim, out_dim)
        
    def forward(self,x):        
        out =self.linear(x)
        return out 

In [5]:
print('Length {} Size {}'.format(len(trainSet), trainSet[0][0].size()))

Length 60000 Size torch.Size([1, 28, 28])


In [6]:
in_dim = 28*28 #input dimensions
out_dim = 10 #output dimensions
model = MultiLogisticModel(in_dim, out_dim) #instantiate model
criterion = nn.CrossEntropyLoss()  #instantiate the loss class
optimiser = torch.optim.SGD(model.parameters(), lr=.1) #instantiate the optimiser class

In [7]:
batchSize = 200
epochs = 1
trainloader = torch.utils.data.DataLoader(dataset=trainSet, batch_size=batchSize, shuffle = True)

In [8]:
for epoch in range(epochs): 

    runningLoss = 0.0
    for i, (images,labels) in enumerate(trainloader):
        images = images.view(-1, 28 * 28)       
        optimiser.zero_grad()
        outputs = model(images)        
        loss = criterion(outputs, labels)
        loss.backward()
        optimiser.step()                      
        runningLoss += loss.item()
      
    print(runningLoss)  

196.61259549856186


In [9]:
predicted = model.forward(images) 
predicted.size()

torch.Size([200, 10])

In [10]:
print('predictions {}'.format(predicted[0]))
print('labels {}'.format(labels[0]))


predictions tensor([-3.7056,  0.0625,  1.0488, -0.0671, -0.3245,  1.4037,  4.0016,
        -3.3711,  2.2218, -1.1201])
labels 6


In [11]:
labels[0:3]

tensor([ 6,  8,  1])

In [12]:
import numpy as np
def successRate(predicted, labels):
    predict = [np.argmax(p.detach().numpy()) for p in predicted]
    actual =  [labels[i].item() for i in range(len(predicted))]
    correct = [i for i, j in zip(predict, actual) if i == j]
    return(len(correct)/(len(predict)))

In [13]:
successRate(predicted, labels)

0.92

In [14]:
testSet = dsets.MNIST(root='./data', train = False, transform=trans.ToTensor(), download=True)
testloader = torch.utils.data.DataLoader(dataset=trainSet,  batch_size = 10000, shuffle = True)

testData = iter(testloader)
images, labels = testData.next()
output = model(images.view(-1, 28 * 28))
successRate(output,labels)

0.886

In [15]:
testSet = dsets.MNIST(root='./data', train = False, transform=trans.ToTensor(), download=True)
testloader = torch.utils.data.DataLoader(dataset=trainSet, batch_size = batchSize, shuffle = True)

testData = iter(testloader)
images, labels = testData.next()
output = model(images.view(-1, 28 * 28))
#successRate(output,labels)
predicted = model.forward(images.view(-1, 28 * 28))   
print(predicted[0:3])
print(labels[0:3])
successRate(predicted,labels)

tensor([[-3.4977,  4.9538, -0.1545,  0.4681, -1.7700,  0.5497,  0.0962,
         -1.0039,  1.2567, -0.7102],
        [-2.1123, -1.0816,  1.0725, -0.7383,  0.3273,  0.1948,  4.3535,
         -1.8619,  0.1997, -0.7896],
        [ 9.4955, -6.7109,  2.8008,  0.1442, -4.8314,  3.1091, -0.1016,
         -1.7326,  1.2769, -3.4950]])
tensor([ 1,  6,  0])


0.905

In [16]:
pred = [np.argmax(p.detach().numpy()) for p in output] 
incorrect = ([i for i, j in zip(pred, labels) if i != j])

In [17]:
images[0].size()

torch.Size([1, 28, 28])

784